In [273]:
from numpy import loadtxt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report, precision_score, accuracy_score
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
import pandas as pd
import matplotlib.pyplot as plt
import csv
import numpy as np
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [274]:
dataset = pd.read_csv("/content/drive/MyDrive/Machine_Learning_Lab_Datasets/nn_data.txt", header=None)
dataset.shape
print(dataset.head())

   0    1   2   3    4     5      6   7  8
0  6  148  72  35    0  33.6  0.627  50  1
1  1   85  66  29    0  26.6  0.351  31  0
2  8  183  64   0    0  23.3  0.672  32  1
3  1   89  66  23   94  28.1  0.167  21  0
4  0  137  40  35  168  43.1  2.288  33  1


In [275]:
from sklearn import preprocessing
minmax_scale = preprocessing.MinMaxScaler().fit(dataset)
scaled_dataset = minmax_scale.transform(dataset)

In [276]:
scaled_dataset

array([[0.35294118, 0.74371859, 0.59016393, ..., 0.23441503, 0.48333333,
        1.        ],
       [0.05882353, 0.42713568, 0.54098361, ..., 0.11656704, 0.16666667,
        0.        ],
       [0.47058824, 0.91959799, 0.52459016, ..., 0.25362938, 0.18333333,
        1.        ],
       ...,
       [0.29411765, 0.6080402 , 0.59016393, ..., 0.07130658, 0.15      ,
        0.        ],
       [0.05882353, 0.63316583, 0.49180328, ..., 0.11571307, 0.43333333,
        1.        ],
       [0.05882353, 0.46733668, 0.57377049, ..., 0.10119556, 0.03333333,
        0.        ]])

In [277]:
dataset

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [278]:
# split into input (X) and output (y) variables
scaled_X = scaled_dataset[:,0:8]
scaled_y = scaled_dataset[:,8]

In [279]:
X = dataset.iloc[:,0:8]
y = dataset.iloc[:,8]

In [280]:
# Defining Number of Folds for K fold validation
from sklearn.model_selection import StratifiedKFold as kfold
folds = kfold(n_splits=10)
folds

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)

In [281]:
# Delcaring two array for storing precision and accuracy for each folds.
precisionScores = []
accuracyScores = []

In [282]:
# The Neural Network Model for K Fold Validation
def ann_model(x_train,y_train,x_test,y_test,epochs,batch_size) :
    model = Sequential();
    model.add(Dense(8, input_shape=(8,), activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile( optimizer = 'adam' , loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.fit(x_train,y_train,epochs=epochs,batch_size = batch_size)
    # _, acuuracy , precision = model.evaluate(x_test,y_test)
    yp = model.predict(x_test)
    y_pred = [] ;
    for element in yp:
        if element > 0.5 :
            y_pred.append(1)
        else :
            y_pred.append(0)
    print(classification_report(y_test,y_pred))
    precisionScores.append(precision_score(y_test, y_pred))
    accuracyScores.append(accuracy_score(y_test, y_pred))
    

In [283]:
# The Neural Network Model without K Fold validation
def ann_model_without_k_fold(X,y,epochs,batch_size) :
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)
    model = Sequential();
    model.add(Dense(8, input_shape=(8,), activation='relu'))
    model.add(Dense(6, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile( optimizer = 'adam' , loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.fit(X_train,y_train,epochs=epochs,batch_size = batch_size)
    yp = model.predict(X_test);
    y_pred = [] ;
    for element in yp:
        if element > 0.5 :
            y_pred.append(1)
        else :
            y_pred.append(0)
    precisionScore = precision_score(y_test, y_pred)
    accuracyScore = accuracy_score(y_test, y_pred)
    return precisionScore,accuracyScore ;

In [284]:
precisionScore,accuracyScore = ann_model_without_k_fold(X,y,100,10)

Epoch 1/100
54/54 [==============================] - 1s 2ms/step - loss: 1.4246 - accuracy: 0.5829
Epoch 2/100
54/54 [==============================] - 0s 2ms/step - loss: 1.0320 - accuracy: 0.6406
Epoch 3/100
54/54 [==============================] - 0s 2ms/step - loss: 0.8967 - accuracy: 0.6536
Epoch 4/100
54/54 [==============================] - 0s 2ms/step - loss: 0.7975 - accuracy: 0.6555
Epoch 5/100
54/54 [==============================] - 0s 2ms/step - loss: 0.7587 - accuracy: 0.6555
Epoch 6/100
54/54 [==============================] - 0s 2ms/step - loss: 0.7261 - accuracy: 0.6667
Epoch 7/100
54/54 [==============================] - 0s 2ms/step - loss: 0.7065 - accuracy: 0.6704
Epoch 8/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6943 - accuracy: 0.6778
Epoch 9/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6824 - accuracy: 0.6741
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6727 - accuracy: 0.6834
Epoch 11/

In [285]:
print("Precision :  %.2f%% " % (precisionScore))
print("Accuracy :  %.2f%% " % (accuracyScore))

Precision :  0.64% 
Accuracy :  0.68% 


In [286]:


precisionScore,accuracyScore = ann_model_without_k_fold(scaled_X,scaled_y,100,10)



Epoch 1/100
54/54 [==============================] - 1s 2ms/step - loss: 0.6920 - accuracy: 0.5419
Epoch 2/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6751 - accuracy: 0.6462
Epoch 3/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6657 - accuracy: 0.6443
Epoch 4/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6591 - accuracy: 0.6443
Epoch 5/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6547 - accuracy: 0.6443
Epoch 6/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6506 - accuracy: 0.6443
Epoch 7/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6472 - accuracy: 0.6443
Epoch 8/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6438 - accuracy: 0.6443
Epoch 9/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6399 - accuracy: 0.6443
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 0.6364 - accuracy: 0.6443
Epoch 11/

In [287]:
print("Precision :  %.2f%% " % (precisionScore))
print("Accuracy :  %.2f%% " % (accuracyScore))

Precision :  0.70% 
Accuracy :  0.79% 


In [288]:
# Implementing with K Fold validation
for train_index, test_index in folds.split(scaled_X,scaled_y):
   X_train, X_test = scaled_X[train_index], scaled_X[test_index]
   y_train, y_test = scaled_y[train_index], scaled_y[test_index]
   ann_model(X_train,y_train,X_test,y_test,100,10)

Epoch 1/100
70/70 [==============================] - 1s 2ms/step - loss: 0.6959 - accuracy: 0.5065
Epoch 2/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6709 - accuracy: 0.6556
Epoch 3/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6559 - accuracy: 0.6512
Epoch 4/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6458 - accuracy: 0.6512
Epoch 5/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6381 - accuracy: 0.6512
Epoch 6/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6320 - accuracy: 0.6512
Epoch 7/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6259 - accuracy: 0.6512
Epoch 8/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6196 - accuracy: 0.6527
Epoch 9/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6126 - accuracy: 0.6541
Epoch 10/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6055 - accuracy: 0.6643
Epoch 11/

In [289]:
precisionScores 

[0.6206896551724138,
 0.7777777777777778,
 0.75,
 0.6153846153846154,
 0.7058823529411765,
 0.75,
 0.8260869565217391,
 0.7407407407407407,
 0.631578947368421,
 0.8260869565217391]

In [290]:
accuracyScores

[0.7402597402597403,
 0.7792207792207793,
 0.7792207792207793,
 0.7272727272727273,
 0.7402597402597403,
 0.7792207792207793,
 0.8441558441558441,
 0.8181818181818182,
 0.7236842105263158,
 0.8552631578947368]

In [291]:
print("Avg Precision :  %.2f%% (+/- %.2f%%)" % (np.mean(precisionScores), np.std(precisionScores)))
print("Avg Accuracy : %.2f%% (+/- %.2f%%)" % (np.mean(accuracyScores), np.std(accuracyScores)))

Avg Precision :  0.72% (+/- 0.08%)
Avg Accuracy : 0.78% (+/- 0.05%)


As we can see , 

*   For Unscaled Dataset ->  **Precision : 0.64 , Accuracy : 0.68** ,
*   For Scaled Dataset Without K FOLD Validation -> **Precision : 0.70, Accuracy : 0.79,**
*   For Scaled Dataset With K FOLD Validation -> **Precision : 0.72 , Accuracy : 0.78**



